# Custom Observers

Observers are at the heart of PyBN, but unfortunately it is not possible to define a recipe for everyones needs, but we built the system flexible enough that anybody can design its own observer. For simplicity of reading of this section we will recurr to a Jupyter Notebook trick to define a class along multiple code blocks. All observers derive from the class Observer.

In [1]:
from pybn.observers import Observer
import numpy as np

For this tutorial we will create an observer that just adds the state of each node in the network at each timestep and the observation is the average of such quantity. The first step is thus making our custom observer to derive from it and define the __init__ method. We will add the tag REQUIRED when the observer must have an implementation of that function.

In [2]:
class MyCustomObserver(Observer):
    #REQUIRED.
    def __init__(self, nodes=1, runs=1):
        # Always call the constructor for the parent class.
        super().__init__(nodes=nodes, runs=runs)
        
        # Observations is used to declare the number and the name of the observations the observer will return.
        # This will also be the name used for the file within the execution module. 
        # Thus, it is important to not repeat the names for the observartions.
        self.observations = ['average_sum']
        
        # Data is where the observer will be storing all the data.
        # This variable is defined in the super class but may be modified if needed, 
        # but the first dimension of this variable must be number of runs.
        # self.data = np.zeros((self.runs, self.nodes))
        
        # Aditional variables may be declared if needed.
        self.table = np.zeros((self.nodes))
        self.table_requires_update = False

We also need to declare a custom method to build this observer from a configuration file.

In [3]:
class MyCustomObserver(MyCustomObserver):
    #REQUIRED.
    @classmethod
    def from_configuration(cls, configuration):
        return cls(
            nodes=configuration['parameters']['nodes'],  
            runs=configuration['execution']['samples'])

We need to declare methods to clear the temporal variables after each run and to reset the observer to is default value. Only custom variables need to be modified.

In [4]:
class MyCustomObserver(MyCustomObserver):
    #REQUIRED.
    def clear(self):
        # This function is called each time the networks starts a new run.
        # Its function is to reset al temporal variables.
        self.table = np.zeros((self.nodes))
        
        # Always call post_clear() at the end of clear.
        self.post_clear()

    #REQUIRED.
    def reset(self):
        # This function is called by the user when it wants to reset the observer to default.
        # Most of the time will be called from network.reset_observers().
        self.table = np.zeros((self.nodes))
        
        # Always call post_reset() at the end of clear.
        # self.data is reseted here.
        self.post_reset()

The most important method is update. This is the data entry point.

In [5]:
class MyCustomObserver(MyCustomObserver):
    #REQUIRED.
    def update(self, state):
        # Iterate through all nodes at time step t.
        for i in range(len(state)):
            # Add the value of that state.
            self.table[i] += state[i]
        
        # Always call post_update() at the end of clear.
        self.post_update()

We also need to define a method that process all the data, for this case we just need to divide the sum and the number of states the observer has seen (self.counter).

In [6]:
class MyCustomObserver(MyCustomObserver):
    #REQUIRED.
    def process_data(self):
        # Average all the values and store it in self.data.
        self.data[self.current_run] = self.table / self.counter 

Finally we just need to define the method observations_to_data to recover the data the observer is calculating. This method helps the observer to parse data and format it back to the you in a readble format. This method needs to be defined for two cases, the average of the network and the per node case. We will also add a few methods that will help the calculations. You may declare as many of these as necessary.

In [7]:
class MyCustomObserver(MyCustomObserver):
    #REQUIRED.
    def observations_to_data(self, observation_name, per_node=False):
        if (observation_name == self.observations[0]):
            return self.observation_0(per_node=per_node)
        # Add as many cases as necessary
        # elif(observation_name == self.observations[1]):
        #     return self.observation_1(per_node=per_node)
        
    #AUXILIAR.
    # Here we are returning the mean and the std of the observation but in general you may define as many different quantities as you need.
    # Just notice that they will be stored / printed as:
    # (Network average) [network_quantity_0, ... ,network_quantity_k]
    # (Per node) [node_0_quantity_0, ... ,node_0_quantity_k, node_1_quantity_0, ..., node_n_quantity_k]
    def observation_0(self, per_node=False):
        if (per_node):
            return np.mean(self.data, axis=0), np.std(self.data, axis=0)
        else:
            return np.mean(self.data), np.std(self.data)

That's it, our custom observer is properly defined.

# Testing the custom observer

In [8]:
from pybn.graphs import uniform_graph
from pybn.networks import BooleanNetwork

Lets define a small network that only perform a few steps in order to visually compare the result of the observer with the state evolution of the network.

In [9]:
nodes = 8
steps = 5
average_connectivity = 3.1

graph = uniform_graph(nodes, average_connectivity)
network = BooleanNetwork(nodes, graph)

In [10]:
MyCustomObserver(nodes=nodes)

Instantiate and attach the observer.

In [11]:
observers = [MyCustomObserver(nodes=nodes)] 
network.attach_observers(observers)

Perform one small execution and print the states.

In [12]:
# Set a random initial state.
network.set_initial_state(observe=True)
print(network.state)
# Perform several steps.
for _ in range(steps):
    network.step(observe=True)
    print(network.state)

[0 0 1 1 1 1 0 0]
[0 1 1 1 1 0 0 1]
[0 0 0 1 1 0 0 0]
[0 0 0 0 1 0 0 1]
[0 0 0 1 1 0 0 1]
[0 0 0 0 1 0 0 1]


Print the observers summary.

In [13]:
# Get observer's summary.    
network.observers_summary()

Network average sum:	0.375 ± 0.341

Network average sum (per node):
0.000 ± 0.000,	0.167 ± 0.000,	0.333 ± 0.000,	0.667 ± 0.000,	1.000 ± 0.000,	
0.167 ± 0.000,	0.000 ± 0.000,	0.667 ± 0.000,	


